# Librerías

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random as rd

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Flatten,Dense,Dropout,BatchNormalization,LSTM,Conv1D,MaxPool1D
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.callbacks import EarlyStopping

plt.rcParams['figure.figsize'] = [8, 6] 

# LCG de 8 bits
Se define la función "LCG" que devuelve una secuencia de números pseudoaleatorios de 8 bits de acuerdo a la fórmula recursiva

$X_{n+1} = \left(aX_n + c\right)\text{mod}\;\mathcal{M}$ 

Donde $a$, $c$ y $\mathcal{M}$ son el multiplicador, incremento y periodo del generador, respectivamente. 
En este caso, se hace $\mathcal{M} = 2^{\text{potencia}}$, donde "potencia" es una variable utilizada para definir el periodo del generador.

In [ ]:
bits = 8
potencia = 20

def LCG(seed,longitud):
    numeros_generados = np.zeros(longitud, dtype = 'int16')

    a = 25214903917
    c = 1
    M = 2 ** potencia
    
    xi = seed
    xf = 0

    numeros_generados[0] = int(((seed%M)/M) * (2 ** bits))

    for i in range(1,longitud):
        xf = (a*xi + c)%M
        xi = xf
  
        numeros_generados[i] = int((xf/M) * (2 ** bits))

    return numeros_generados

Se crea el arreglo "muestras" que será utilizado para construir los conjuntos de machine learning. La semilla se elige aleatoriamente. La secuencia pseudoaleatoria tiene como longitud el periodo del generador.

In [ ]:
muestras = LCG(seed = rd.randint(1,1000), longitud = 2 ** potencia)
prob_adivinar = 1/(2 ** bits)

n, bins, patches = plt.hist(muestras, bins = 2 ** bits, density = 1, color = 'green')
plt.axhline(prob_adivinar, color = 'red', label = 'y = ' + str(prob_adivinar))
plt.xlabel('Número de 8 bits')
plt.ylabel('Probabilidad')
plt.legend()
plt.grid()
plt.show()

# Conjuntos de ML
Se construyen los conjuntos de entrenamiento y prueba o test. Como input, se utilizan secuencias cuya longitud es dada por la variable "longitud_input". Dichas secuencias se codifican con vectores one-hot. Por otro lado, "porcentaje" determina qué fracción de la muestra será para entrenar y cuanto se utilizará para el test final.

In [ ]:
longitud_input = 13
porcentaje = 0.8

longitud_entrenamiento = int(porcentaje * 2 ** potencia)
longitud_test = int((1 - porcentaje) * 2 ** potencia - (longitud_input + 1))

inputs_entrenamiento = np.zeros((longitud_entrenamiento, longitud_input, 2 ** bits), dtype = 'int8')
outputs_entrenamiento = np.zeros((longitud_entrenamiento, 2 ** bits), dtype = 'int8')

inputs_test = np.zeros((longitud_test, longitud_input, 2 ** bits), dtype = 'int8')
outputs_test = np.zeros((longitud_test, 2 ** bits), dtype = 'int8')

for i in range(0, longitud_entrenamiento):
    for j in range(0, longitud_input):
        inputs_entrenamiento[i, j, muestras[i + j]] = 1 
    
    outputs_entrenamiento[i, muestras[i + longitud_input]] = 1
    
for i in range(longitud_entrenamiento, longitud_entrenamiento + longitud_test):
    for j in range(0, longitud_input):
        inputs_test[i - longitud_entrenamiento, j, muestras[i + j]] = 1
        
    outputs_test[i - longitud_entrenamiento, muestras[i + longitud_input]] = 1
    
print(inputs_entrenamiento.shape)
print(outputs_entrenamiento.shape)
print(inputs_test.shape)
print(outputs_test.shape)

del muestras #Se borra muestras para ahorrar espacio en memoria

# Modelo de red recurrente convolucional (RCNN)
Se construye el modelo de red con keras.

In [ ]:
input_data_shape = (longitud_input, 2 ** bits)
activacion_conv = 'relu'
activacion_lstm = 'tanh'
activacion_oculta = 'relu'
activacion_output = 'softmax'

RCNN = Sequential()

RCNN.add(Conv1D(input_shape = input_data_shape, filters = 64, kernel_size = 5, activation = activacion_conv, padding = 'same', name = 'capa_conv_1'))
RCNN.add(MaxPool1D(pool_size = 2, name = 'capa_pooling'))
RCNN.add(BatchNormalization())
RCNN.add(LSTM(units = 128, activation = activacion_lstm, return_sequences = False, name = 'capa_LSTM'))
RCNN.add(BatchNormalization())
RCNN.add(Dense(units = 64, activation = activacion_oculta, name = 'capa_oculta'))
RCNN.add(BatchNormalization())
RCNN.add(Dense(2 ** bits, activation = activacion_output, name = 'capa_outputs'))

RCNN.summary()

# Entrenamiento
Se entrena el modelo de machine learning con el conjunto de entrenamiento. Algunos hiperparámetros como la tasa de entrenamiento y el número de epochs se establecen al principio de la celda. La función de costo es "categorical_crossentropy". Adicionalmente, se utiliza un 10% de la data de entrenamiento como data de validación.

Cabe mencionar que se aplica un EarlyStopping, el cual detiene el entrenamiento cuando el costo de validación no mejora en 10 epochs consecutivos. Al final del entrenamiento, se guarda la red con los parámetros que minimizaron el costo de validación.

In [ ]:
tasa_entrenamiento = 0.001
numero_epochs = 200
tamanio_minilote = 1024

optimizador = Adam(learning_rate = tasa_entrenamiento)
funcion_costo = 'categorical_crossentropy'
metrica = ['accuracy']
callback = EarlyStopping(monitor = 'val_loss', patience = 10, restore_best_weights=True)

RCNN.compile(optimizer = optimizador, loss = funcion_costo, metrics = metrica)
entrenamiento = RCNN.fit(inputs_entrenamiento, outputs_entrenamiento, batch_size = tamanio_minilote,
                        callbacks = [callback], validation_split= 0.1, epochs = numero_epochs)

# Gráficas
Se observa la evolución de la red a través de gráficas de costo y precisión tanto sobre el conjunto de entrenamiento como el conjunto de validación.

In [ ]:
plt.plot(entrenamiento.history['accuracy'], label = 'Entrenamiento')
plt.plot(entrenamiento.history['val_accuracy'], label = 'Validación')
plt.axhline(prob_adivinar, color = 'red', label = 'P. Adivinar')
plt.ylabel('Precisión')
plt.xlabel('Epoch')
plt.grid()
plt.legend()
plt.show()

plt.plot(entrenamiento.history['loss'], label = 'Entrenamiento')
plt.plot(entrenamiento.history['val_loss'], label = 'Validación')
plt.ylabel('Costo')
plt.xlabel('Epoch')
plt.grid()
plt.legend()
plt.show()

# Evaluación
Se evalúa la red mediante el conjunto de prueba o test para observar si el modelo pudo generalizarse.

In [ ]:
costo_test, precision_test = RCNN.evaluate(inputs_test, outputs_test)

print("El costo sobre el conjunto de prueba es: " + str(costo_test))
print("La precisión sobre el conjunto de prueba es: " + str(precision_test))